<a href="https://colab.research.google.com/github/rohansanjay/LookDeep/blob/master/MobileNetV2_10_06_(Version_2)_%2B_OD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub
from tensorflow.keras import datasets, layers, models

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Image imports
from zipfile import ZipFile
import matplotlib.pyplot as plt
import pandas as pd

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

tranch = 1
labels_path = '/content/drive/My Drive/M499_F20_Data/tranch'+str(tranch)+'_labels.csv'
pictures_path = '/content/drive/My Drive/M499_F20_Data/persons-posture-tranch'+str(tranch)+'.zip'

labels = pd.read_csv(labels_path)
zip_file = ZipFile(pictures_path)

file_list = [obj.filename for obj in zip_file.infolist()]
file_list_simple = [name.split('/')[-1] for name in file_list]

2.3.0
The following GPU devices are available: 


In [ ]:
names = pd.DataFrame({'file_path': file_list, 'file_name': file_list_simple})
names.head();

df = pd.merge(names, labels, on = 'file_name')
print(len(names), len(labels), len(df))
df.head()

16940 16922 16921


,file_path,file_name,how_many,primary_posture,primary_occluded,staff_patient_other
0,home/tyler/git/darknet/data/persons/yolo/v4/e2...,e2_0209_cat-1_002.jpg,One,Sitting,No,Patient
1,home/tyler/git/darknet/data/persons/yolo/v4/SU...,SU-O-390_cat-1_000.png,Two,Sitting,No,Patient
2,home/tyler/git/darknet/data/persons/yolo/v4/SU...,SU-O-602_cat-1_005.jpg,One,Standing,Yes,Staff
3,home/tyler/git/darknet/data/persons/yolo/v4/SU...,SU-O-105_cat-1_011.png,Two,Unknown,Yes,NaN
4,home/tyler/git/darknet/data/persons/yolo/v4/SU...,SU-O-83_cat-1_004.png,None,NaN,NaN,NaN


In [ ]:
one_person = df[df['how_many'] == 'One']
one_person = one_person.dropna(subset=['primary_posture'])
one_person = one_person[one_person['primary_occluded'] == 'No']
one_person = one_person[one_person['primary_posture'] != 'Unknown']
one_person = one_person.reset_index(drop=True)
categories = {'Sitting': 0, 'Standing': 1, 'Lying': 2}
one_person['primary_posture_n'] = one_person['primary_posture'].map(categories)
one_person.head()

,file_path,file_name,how_many,primary_posture,primary_occluded,staff_patient_other,primary_posture_n
0,home/tyler/git/darknet/data/persons/yolo/v4/e2...,e2_0209_cat-1_002.jpg,One,Sitting,No,Patient,0
1,home/tyler/git/darknet/data/persons/yolo/v4/ma...,maxresdefault_cat-1_000.jpg,One,Standing,No,Staff,1
2,home/tyler/git/darknet/data/persons/yolo/v4/e2...,e2_0128_cat-1_003.jpg,One,Standing,No,Staff,1
3,home/tyler/git/darknet/data/persons/yolo/v4/30...,30bca933-c73d-4831-8a96-6e738a3a7490_cat-1_004...,One,Standing,No,Staff,1
4,home/tyler/git/darknet/data/persons/yolo/v4/5e...,5ea1a58e-e505-47dc-a0d1-410e671546fd_cat-1_001...,One,Standing,No,Staff,1


In [ ]:
one_person.shape

(6700, 7)

OD pipeline

In [ ]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


# def download_and_resize_image(filepath, new_width=256, new_height=256, display=False):
def download_and_resize_image(filepath, display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  pil_image = Image.open(zip_file.open(filepath))
  #pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=1,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin

def crop_image(image,
                ymin,
                xmin,
                ymax,
                xmax,
                color,
                font,
                thickness=1,
                display_str_list=()):
  """Crop image around bounding"""

  im_width, im_height = image.size
  #print(im_width, im_height)
  
  cropped = image.crop((xmin * im_width, ymin * im_height, xmax * im_width, ymax * im_height))

  return cropped


def draw_boxes(image, boxes, class_names, scores, max_boxes=5, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  ymin, xmin, ymax, xmax = tuple(boxes)
  display_str = "{}: {}%".format(class_names.decode("ascii"),
                                  int(100 * scores))
  print(display_str)
  color = colors[hash(class_names) % len(colors)]
  image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
  
  # draw_bounding_box_on_image(
  #     image_pil,
  #     ymin,
  #     xmin,
  #     ymax,
  #     xmax,
  #     color,
  #     font,
  #     display_str_list=[display_str])
  
  crop = crop_image(
      image_pil,
      ymin,
      xmin,
      ymax,
      xmax,
      color,
      font,
      display_str_list=[display_str])
  
  crop = np.array(crop)
      
  return crop

def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}


  b = bytes('Person', 'utf-8')

  ind = np.where(result['detection_class_entities'] == b)

  if len(ind[0]) == 0:
    return img.numpy()

  ind = ind[0][0]

  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"][ind],
      result["detection_class_entities"][ind], result["detection_scores"][ind])
  
  return image_with_boxes

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# IMPORTANT
# num = np.random.randint(0,len(one_person))
# downloaded_image_path = download_and_resize_image(one_person.iloc[num].file_path, True)

Model

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image

train_ims = []
train_labels = []
test_ims = []
test_labels = []
train_num = 0
test_num = 0
i = 0

while train_num < 20:
  downloaded_image_path = download_and_resize_image(one_person.iloc[i].file_path, False)
  cropped = run_detector(detector, downloaded_image_path)

  # ratio check - skip the picture
  r = cropped.shape[0] / cropped.shape[1]
  if r > 2 or r < 0.5:
    i += 1
    continue

  im = cv2.resize(cropped, (256, 256))
  train_ims.append(im)
  train_labels.append(one_person.iloc[i].primary_posture_n)

  train_num += 1
  i += 1

while test_num < 0:
  downloaded_image_path = download_and_resize_image(one_person.iloc[i].file_path, False)
  cropped = run_detector(detector, downloaded_image_path)

  # ratio check - skip the picture
  r = cropped.shape[0] / cropped.shape[1]
  if r > 2 or r < 0.5:
    i += 1
    continue

  im = cv2.resize(cropped, (256, 256))
  test_ims.append(im)
  test_labels.append(one_person.iloc[i].primary_posture_n)

  test_num += 1
  i += 1

Image downloaded to /tmp/tmpe3yzdd_m.jpg.
Person: 14%
Image downloaded to /tmp/tmpwq_evx_s.jpg.
Person: 39%
Image downloaded to /tmp/tmprviopag8.jpg.
Person: 22%
Image downloaded to /tmp/tmpz__5s336.jpg.
Person: 19%
Image downloaded to /tmp/tmph7vo0y7x.jpg.
Person: 11%
Image downloaded to /tmp/tmprurdxecn.jpg.
Person: 18%
Image downloaded to /tmp/tmpchoissnk.jpg.
Person: 51%
Image downloaded to /tmp/tmpmiuwiqld.jpg.
Person: 21%
Image downloaded to /tmp/tmpbqruuaan.jpg.
Person: 39%
Image downloaded to /tmp/tmpsnu9kys1.jpg.
Person: 45%
Image downloaded to /tmp/tmpdo70y_2t.jpg.
Person: 30%
Image downloaded to /tmp/tmp9iyi0si2.jpg.
Person: 10%
Image downloaded to /tmp/tmp59x93ql2.jpg.
Person: 75%
Image downloaded to /tmp/tmpzykx5dvy.jpg.
Person: 30%
Image downloaded to /tmp/tmp51no9jtf.jpg.
Person: 59%
Image downloaded to /tmp/tmp1izdtt7l.jpg.
Person: 22%
Image downloaded to /tmp/tmp0rrg3647.jpg.
Person: 28%
Image downloaded to /tmp/tmpdun4pmr_.jpg.
Person: 37%
Image downloaded to /tmp/tmp

KeyboardInterrupt: ignored

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 444, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 642, in _abort_queue
    ide

In [ ]:
train_ims = np.array( train_ims ) / 255
train_labels = np.array( train_labels )
test_ims = np.array( test_ims ) / 255
test_labels = np.array( test_labels )

NameError: ignored

In [ ]:
train_ims.shape

(1500, 256, 256, 3)

In [ ]:
train_labels = tf.keras.utils.to_categorical( train_labels , num_classes=3 )
test_labels = tf.keras.utils.to_categorical( test_labels , num_classes=3 )

In [ ]:
from keras.applications import MobileNetV2
from keras import layers, optimizers

In [ ]:
base_model = MobileNetV2(weights='imagenet',include_top=False, input_shape=(256, 256, 3)) 
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024,activation='relu')(x) 
x = layers.Dense(1024,activation='relu')(x) 
x = layers.Dense(512,activation='relu')(x) 
preds = layers.Dense(3, activation='softmax')(x) 

model = models.Model(inputs=base_model.input,outputs=preds) 

In [ ]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(
    train_ims, train_labels,
    epochs=10,
    batch_size=10,
    callbacks=None,
)

Epoch 1/10
150/150 [==============================] - 284s 2s/step - loss: 0.9502 - accuracy: 0.5480
Epoch 2/10
150/150 [==============================] - 286s 2s/step - loss: 0.8697 - accuracy: 0.5887
Epoch 3/10
150/150 [==============================] - 287s 2s/step - loss: 0.8034 - accuracy: 0.6087
Epoch 4/10
150/150 [==============================] - 286s 2s/step - loss: 0.7960 - accuracy: 0.6327
Epoch 5/10
150/150 [==============================] - 286s 2s/step - loss: 0.7012 - accuracy: 0.6707
Epoch 6/10
150/150 [==============================] - 283s 2s/step - loss: 0.6953 - accuracy: 0.6960
Epoch 7/10
150/150 [==============================] - 286s 2s/step - loss: 0.6341 - accuracy: 0.7413
Epoch 8/10
150/150 [==============================] - 288s 2s/step - loss: 0.5535 - accuracy: 0.7720
Epoch 9/10
150/150 [==============================] - 290s 2s/step - loss: 0.4942 - accuracy: 0.8127
Epoch 10/10
150/150 [==============================] - 289s 2s/step - loss: 0.4453 - accura

In [ ]:
model.evaluate(test_ims, test_labels)

16/16 [==============================] - 21s 1s/step - loss: 0.9015 - accuracy: 0.6160


[0.9015434384346008, 0.6159999966621399]

- what size to resize to
- training over all 3 tranches
- min/max pixel size
- preprocessed/untouched test sets
- model architecture 